In [1]:
import os

import time

import numpy as np

import pandas as pd

import tensorflow as tf
import tensorflow_probability as tfp

import networkx as nx

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import matplotlib.cm as cm

# Set font properties for ticks
plt.rcParams['xtick.labelsize'] = 12  # Font size of x-axis tick labels
plt.rcParams['ytick.labelsize'] = 12  # Font size of y-axis tick labels

# Set font properties for labels
plt.rcParams['axes.labelsize'] = 15  # Font size of axis labels

# Set font properties for title
plt.rcParams['axes.titlesize'] = 17  # Font size of titlea

2024-12-18 09:57:05.903870: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-18 09:57:06.021831: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 09:57:06.021956: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 09:57:06.022433: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-18 09:57:06.085605: I tensorflow/core/platform/cpu_feature_g

In [2]:
def confidence_ellipse(cov, centroid, ax, n_std=2.0, facecolor='none', **kwargs):

    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the standard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = centroid[0]

    # calculating the standard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = centroid[1]

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [3]:
os.chdir("Scripts/")
%run -i model.py
%run -i CAL.py
os.chdir("../")

# Spatial model

Consider an individual-based SIS model with individual-specific covariates $w_n = [l_n, c_n]$ where $l_n$ is the location in space of each individual and $c_n = [1 \quad \xi_n]^\top$, with $\xi_n$ standard Gaussian random variable. About $l_n$ each individual is uniformly assigned to one of $10$ cities, their location is then drawn from a bivariate Gaussian with mean and variance-covariance matrix that is specific for the considered city.

Then consider the following individual-specific initial distribution, transition kernel and emission distribution:
$$
    p_0(w_n) 
    = 
    \begin{bmatrix} 
    1-p_{0}\\ p_{0} 
    \end{bmatrix},
$$
$$
    \mathbf{\eta}_{n,t-1}
    =
    \frac{1}{N} \sum_{k \in [N]} 
    \begin{bmatrix}
    0\\
     \frac{\exp\left\{-\frac{\|l_n-l_{k}\|^2}{2\phi}\right\}}{\sqrt{2 \pi \phi}} 
    \end{bmatrix}^\top \mathbf{x}_{k,t-1},
$$
$$
K_{\mathbf{\eta}_{n,t-1}}(w_n) 
= 
\begin{bmatrix} 
\exp{ \left ( -\frac{\beta}{1 + \exp({-c_n^\top b_S)}} \mathbf{\eta}_{n,t-1} - \epsilon \right ) }
&
1 - \exp{ \left ( -\frac{\beta}{1 + \exp({-c_n^\top b_S)}}\mathbf{\eta}_{n,t-1} - \epsilon \right ) }
\\ 
1 - \exp{ \left ( -\frac{1}{1 + \exp{(-c_n^\top b_R)}} \right )}
&
\exp{ \left ( -\frac{1}{1 + \exp{(-c_n^\top b_R)}} \right )}
\end{bmatrix},
$$
$$
    G(w_n) 
    =
    \begin{bmatrix}
    1 - q_S 
    & 
    q_S q_{Se}  
    & 
    q_S (1-q_{Se})\\
    1 - q_I 
    & 
    q_I (1- q_{Sp}) 
    & 
    q_I q_{Sp}
    \end{bmatrix}.
$$

In [6]:
parameters = {"prior_infection":tf.convert_to_tensor([1-0.01, 0.01], dtype = tf.float32),
              "beta_l":tf.convert_to_tensor([-2.0, +2.0], dtype = tf.float32),
              "beta_g":tf.convert_to_tensor([-1.0, -1.0], dtype = tf.float32),
              "log_phi":tf.math.log(
                tf.convert_to_tensor([2.0], dtype = tf.float32)),
              "log_chi":tf.math.log(
                tf.convert_to_tensor([50.0], dtype = tf.float32)),
              "logit_sensitivity":logit(
                tf.convert_to_tensor([0.9], dtype = tf.float32)),
              "logit_specificity":logit(
                tf.convert_to_tensor([0.95], dtype = tf.float32)),
              "logit_prob_testing":logit(
                tf.convert_to_tensor([0.2, 0.5], dtype = tf.float32)),
		"log_epsilon":tf.math.log(tf.convert_to_tensor([0.001], dtype = tf.float32)),}

T = 200
n_covergage = 100
n_gradient_steps = 1000
n_trial = 10

In [7]:
N_pop = 1000
covariates = tf.convert_to_tensor(np.load("Data/SpatialInference/Input/covariates.npy"), dtype = tf.float32)[:N_pop,:]
locations  = tf.convert_to_tensor(np.load("Data/SpatialInference/Input/locations.npy"), dtype = tf.float32)[:N_pop,:]
centroids  = tf.convert_to_tensor(np.load("Data/SpatialInference/Input/centroids.npy"), dtype = tf.float32)
cities_std = tf.convert_to_tensor(np.load("Data/SpatialInference/Input/cities_std.npy"), dtype = tf.float32)

In [ ]:
SIS = spatial_SIS(locations, covariates)

start = time.time()
X, Y = simulator(SIS, parameters, T)
print(time.time()-start)

fig, ax = plt.subplots(2, 4, figsize = (10, 5), sharex=True, sharey=True)

for i in range(2):
	for j in range(4):

		t = 10*(i*4+j)

		infected = tf.where(X[t,0,:,1]==1)[:,0]

		ax[i, j].set_title("Time "+str(t))
		ax[i, j].scatter(locations[:,0], locations[:,1], color = "green", s = 20)
		ax[i, j].scatter(tf.gather(locations[:,0], infected),
					tf.gather(locations[:,1], infected), color = "red", s = 20)
		ax[i, j].scatter(centroids[:,0], centroids[:,1], marker="*", color = "black", s = 10)

		if j==0:
			for city in range(tf.shape(cities_std)[0]):
				confidence_ellipse(tf.linalg.diag(cities_std[city,:]), centroids[city,:], ax[i, j], edgecolor='black')

plt.savefig('Figures/spatial_illustration.png', format='png', dpi=100, bbox_inches='tight')

In [ ]:
learning_parameters = { "beta_l": 2, 
		"beta_g": 2, 
		"log_phi": 1, 
		"log_chi": 1, 
		"logit_prob_testing": 2
		}


ylabel_dict = { "beta_l" : "$\\beta_{I}$", 
		"beta_g" : "$\\beta_{R}$", 
		"log_phi": "$\\log(\\phi)$", 
		"log_chi": "$\\log(\\chi)$", 
		"logit_prob_testing": "$logit(q_T)$"
		}

colors_list         = ["blue", "black", "yellow"]

N_list = [500, 1000, 2000]

fig, ax = plt.subplots(len(learning_parameters)+1, len(N_list), figsize = (len(N_list)*4, (len(learning_parameters)+1)*2), sharex=True)

# ax[0,0].set_ylabel("normalized loss")
# ax[1,0].set_ylabel(r"$\log(\phi)$")
# ax[2,0].set_ylabel(r"$\beta_{\gamma}$")

# ax[len(learning_parameters),0].set_xlabel("iterations")
# ax[len(learning_parameters),1].set_xlabel("iterations")
# ax[len(learning_parameters),2].set_xlabel("iterations")

for n_index in range(len(N_list)):
	N = N_list[n_index]

	ax[0,n_index].set_title(r"$N=$"+str(N))

	output_path = "Data/SpatialInference/Output/"+str(N)+"/"
	true_loss_file_name        = "spatial_inference_"+str(N)+"_true_loss.npy"
	optim_loss_file_name       = "spatial_inference_"+str(N)+"_optim_loss.npy"
	optim_parameters_file_name = "spatial_inference_"+str(N)+"_optim_parameters.npy"

	true_loss        = np.load(output_path+true_loss_file_name,        allow_pickle=False)
	optim_loss       = np.load(output_path+optim_loss_file_name,       allow_pickle=False)
	optim_parameters = np.load(output_path+optim_parameters_file_name, allow_pickle=True)[()]

	index = np.unique(np.where(np.all(optim_loss!=0, axis = -1))[0])

	for i in range(n_covergage):

		if i in index:

			trial_index = np.nanargmin(optim_loss[i,:,-1])

			ax[0, n_index].plot(optim_loss[i,trial_index,:]/true_loss[i], color = "blue", alpha = 0.1)
			ax[0, n_index].axhline(true_loss[i]/true_loss[i], color = "red", alpha = 0.1)
			
			if n_index == 0:
				ax[0, n_index].set_ylabel("loss")	

			counter = 0
			for key in learning_parameters.keys():
				for j in range(learning_parameters[key]):

					ax[counter+1, n_index].plot(optim_parameters[key][i,trial_index,:, j], color = colors_list[j], alpha = 0.1)
					ax[counter+1, n_index].axhline(parameters[key][j].numpy(), color = "red", alpha = 0.1)

				if n_index == 0:
					ax[counter+1, n_index].set_ylabel(ylabel_dict[key])	

				if (counter+1) == len(learning_parameters.keys()):
					ax[counter+1, n_index].set_xlabel("grad steps")
				
				counter = counter +1

plt.savefig('Figures/spatial_inference.png', format='png', dpi=100, bbox_inches='tight')

In [ ]:
# fig, ax = plt.subplots(2, len(learning_parameters), figsize = ((len(learning_parameters))*4, 2*2))

table_dict = {}
counter = 0
for key in learning_parameters.keys():

	table_dict[key] = []

	for j in range(learning_parameters[key]):
		optim_param_list = []
		for n_index in range(len(N_list)):
			N = N_list[n_index]

			output_path = "Data/SpatialInference/Output/"+str(N)+"/"
			true_loss_file_name        = "spatial_inference_"+str(N)+"_true_loss.npy"
			optim_loss_file_name       = "spatial_inference_"+str(N)+"_optim_loss.npy"
			optim_parameters_file_name = "spatial_inference_"+str(N)+"_optim_parameters.npy"

			true_loss        = np.load(output_path+true_loss_file_name,        allow_pickle=False)
			optim_loss       = np.load(output_path+optim_loss_file_name,       allow_pickle=False)
			optim_parameters = np.load(output_path+optim_parameters_file_name, allow_pickle=True)[()]

			index = np.unique(np.where(np.all(optim_loss!=0, axis = -1))[0])

			trial_index = np.nanargmin(optim_loss[index,...,-1], axis =1)
			to_append = np.stack([optim_parameters[key][trial,trial_index[trial],-1,j] for trial in index])
			# to_append = optim_parameters[key][2,:,-1,j]

			optim_param_list.append(tf.gather(to_append, tf.where(tf.math.is_nan(to_append)==False))[:,0])

		table_dict[key].append(optim_param_list)
		# ax[j,counter].boxplot(optim_param_list, showmeans=True)
		# ax[j,counter].axhline(parameters[key][j].numpy(), color = "red", alpha = 0.1)

	counter = counter +1

In [ ]:
from tabulate import tabulate

In [ ]:
# assign data
table = []

for key in learning_parameters.keys():
	for j in range(learning_parameters[key]):
		row = []
		row.append(ylabel_dict[key]+" = "+str(np.round(parameters[key][j].numpy(), 3)))
		for n_index in range(len(N_list)):

			row.append(str(np.round(np.nanmean(table_dict[key][j][n_index]), 3))+"+/-"+str(np.round(np.nanstd(table_dict[key][j][n_index]), 3)))
		
		table.append(row)
 
# create header
head = ["Parameter", "N=100", "N=400", "N=2000"]
 
# display table
print(tabulate(table, headers=head, tablefmt="latex_raw"))

# Graph model

Consider now an individual-based model with individual-specific covariates $w_n$ given by $(l_n, c_n)$ where $c_n$ is as in Section \ref{sec:exp_spatial_model} while $l_n \in [5]$ is a number between $1$ and $5$ representing the community the individual is assigned to, and we denote with $C_i$ the number of individuals in community $i$, i.e. $C_i=\sum_{n \in [N]}\mathbb{I}(l_n=i)$. Specifically, we draw the probabilities of being assign to the different communities from a Dirichlet distribution with concentration parameters that are drawn from a folded Normal with mean $10$ and standard deviation $5$.

Then we consider the following individual-specific initial distribution, transition kernel and emission distribution:
$$
    p_0(w_n,\theta) 
    = 
    \begin{bmatrix} 
    1-p_{0}\\ p_{0} 
    \end{bmatrix},
$$
$$
    \eta_{t-1}^N(w_n,\theta)
    =
    \frac{1}{N} \sum_{\tilde{n} \in [N]\setminus n} \begin{bmatrix}
    0\\
    \frac{N}{C_{l_{\tilde{n}}}} B(l_n,l_{\tilde{n}},\phi)
    \end{bmatrix}^\top x_{t-1}^N(w_{\tilde{n}}),
$$
$$
    K_{\eta_{t-1}^N(w_n,\theta)}(w_n,\theta) 
    = 
    \begin{bmatrix} 
    1 - \frac{\eta_{t-1}^N(w_n,\theta) +\epsilon }{1 + \exp{(-\beta_{I}^\top c_n)}}
    & 
    \frac{\eta_{t-1}^N(w_n,\theta) +\epsilon }{1 + \exp{(-\beta_{I}^\top c_n)}}\\ 
    \frac{1}{1 + \exp{(-\beta_{R}^\top c_n)}}
    & 
    1- \frac{1}{1 + \exp{(-\beta_{R}^\top c_n)}}
    \end{bmatrix},
$$
$$
    G(w_n,\theta) 
    =
    \begin{bmatrix}
    1 - q_T^{(1)} 
    & 
    q_T^{(1)} q_{Se}  
    & 
    q_T^{(1)} (1-q_{Se})\\
    1 - q_T^{(2)} 
    & 
    q_T^{(2)} (1- q_{Sp}) 
    & 
    q_T^{(2)} q_{Sp}
    \end{bmatrix},
$$
where $p_0$ is the probability of being infected at the beginning of the epidemic, $B(i,j,\phi)= \prod_{k=1}^3 \exp ( -10 k \phi\mathbb{I}(|{i-j}|=k) )$ is a function that tells us the probability of a contact between $i$ and $j$, $\epsilon$ is an environmental effect, $\beta_I$ are the logistic regression coefficients for the infectious rate, $\beta_R$ are the logistic regression coefficients for the recovery rate, $q_T$ is the vector of reporting probabilities with $q_T^{(1)}$ being reported as a susceptible and $q_T^{(2)}$ as an infected, $q_{Se}$ and $q_{Sp}$ are the sensitivity and specificity of the test. Essentially at each time step each individual interacts with the others according to a stochastic block model with a probability matrix that is given by $(B(i,j,\phi))_{i,j \in [5]}$.

In [ ]:
parameters = {"prior_infection":tf.convert_to_tensor([1-0.01, 0.01], dtype = tf.float32),
              "beta_l":tf.convert_to_tensor([-1.0, +2.0], dtype = tf.float32),
              "beta_g":tf.convert_to_tensor([-1.0, -1.0], dtype = tf.float32),
	      "log_graph":tf.math.log(tf.convert_to_tensor([0.1], dtype = tf.float32)),
              "logit_sensitivity":logit(
                tf.convert_to_tensor([0.9], dtype = tf.float32)),
              "logit_specificity":logit(
                tf.convert_to_tensor([0.95], dtype = tf.float32)),
              "logit_prob_testing":logit(
                tf.convert_to_tensor([0.2, 0.5], dtype = tf.float32)),
	      "log_epsilon":tf.math.log(tf.convert_to_tensor([0.001], dtype = tf.float32)),}


T = 200
n_covergage = 100
n_gradient_steps = 1000
n_trial = 10

In [ ]:
N_pop = 10000
covariates  = tf.convert_to_tensor(np.load("Data/GraphInference/Input/covariates.npy"), dtype = tf.float32)[:N_pop,:]
communities = tf.convert_to_tensor(np.load("Data/GraphInference/Input/communities.npy"), dtype = tf.float32)[:N_pop,:]

SIS = sbm_SIS(communities, covariates)

start = time.time()
X, Y = simulator(SIS, parameters, T)
print(time.time()-start)
X, Y = X[:,0,...], Y[:,0,...]

B = SIS.B_matrix(parameters)
nodes = range(B.shape[0])
G = nx.Graph()
for i in nodes:
    for j in nodes:
        if i > j and B.numpy()[i, j]!=0:
            G.add_edge(i, j, weight=10*B.numpy()[i, j])

fig, ax = plt.subplots(1, 1, figsize = (2, 2))      
ax.imshow(B)

In [ ]:
population = tf.reduce_sum(X, axis = 1)

fig, ax = plt.subplots(2, 4, figsize = (15, 7), sharex=True, sharey=True)

jump_list = [5, 10]
for i in range(2):
	for j in range(4):

		t = jump_list[i]*(i*4+j)

		ax[i, j].set_title("Time "+str(t))
		
		current_state = tf.einsum("tns,nc->tcs", X, SIS.communities)[t,...]
		percentage    = current_state/tf.reduce_sum(current_state, axis = 1, keepdims=True)

		pos = nx.spring_layout(G, seed = 10)  # Positions for all nodes
		weight_labels = nx.get_edge_attributes(G, 'weight')

		color_map = ["green" for node in G.nodes] 
		nx.draw_networkx_nodes( G, pos, 
					node_color = color_map, node_shape='o', node_size = 0.25*tf.reduce_sum(SIS.communities, axis = 0),
					ax = ax[i,j])

		color_map = ["red" for node in G.nodes] 
		nx.draw_networkx_nodes( G, pos, 
			 		node_color=color_map, node_shape='o', node_size = 0.25*tf.reduce_sum(SIS.communities, axis = 0)*percentage[:,1],
					ax = ax[i,j])
		
		nx.draw_networkx_edges( G, pos, 
			 		edgelist = weight_labels.keys(), width=list(weight_labels.values()), edge_color='black', alpha=0.5,
					ax = ax[i,j])
		
plt.savefig('Figures/graph_illustration.png', format='png', dpi=100, bbox_inches='tight')

In [ ]:
learning_parameters = { "beta_l": 2, 
			"beta_g": 2, 
			"logit_prob_testing": 2, 
			"log_graph": 1}

ylabel_dict = { "beta_l": "$\\beta_{I}$", 
		"beta_g": "$\\beta_{R}$", 
		"logit_prob_testing": "$logit(q_T)$", 
		"log_graph": "$\\log(\\phi)$"}

colors_list         = ["blue", "black", "yellow"]

N_list = [500, 5000, 50000]

fig, ax = plt.subplots(len(learning_parameters)+1, len(N_list), figsize = (len(N_list)*4, (len(learning_parameters)+1)*2), sharex=True)

# ax[0,0].set_ylabel("normalized loss")
# ax[1,0].set_ylabel(r"$\log(\phi)$")
# ax[2,0].set_ylabel(r"$\beta_{\gamma}$")

# ax[len(learning_parameters),0].set_xlabel("iterations")
# ax[len(learning_parameters),1].set_xlabel("iterations")
# ax[len(learning_parameters),2].set_xlabel("iterations")

for n_index in range(len(N_list)):
	N = N_list[n_index]

	ax[0,n_index].set_title(r"$N=$"+str(N))

	output_path = "Data/GraphInference/Output/"+str(N)+"/"
	true_loss_file_name        = "graph_inference_"+str(N)+"_true_loss.npy"
	optim_loss_file_name       = "graph_inference_"+str(N)+"_optim_loss.npy"
	optim_parameters_file_name = "graph_inference_"+str(N)+"_optim_parameters.npy"

	true_loss        = np.load(output_path+true_loss_file_name,        allow_pickle=False)
	optim_loss       = np.load(output_path+optim_loss_file_name,       allow_pickle=False)
	optim_parameters = np.load(output_path+optim_parameters_file_name, allow_pickle=True)[()]

	index = np.unique(np.where(np.all(optim_loss!=0, axis = -1))[0])

	for i in range(n_covergage):

		if i in index:

			trial_index = np.nanargmin(optim_loss[i,:,-1])

			ax[0, n_index].plot(optim_loss[i,trial_index,:]/true_loss[i], color = "blue", alpha = 0.1)
			ax[0, n_index].axhline(true_loss[i]/true_loss[i], color = "red", alpha = 0.1)
			if n_index == 0:
				ax[0, n_index].set_ylabel("loss")	

			counter = 0
			for key in learning_parameters.keys():
				for j in range(learning_parameters[key]):

					ax[counter+1, n_index].plot(optim_parameters[key][i,trial_index,:, j], color = colors_list[j], alpha = 0.1)
					ax[counter+1, n_index].axhline(parameters[key][j].numpy(), color = "red", alpha = 0.1)

				if n_index == 0:
					ax[counter+1, n_index].set_ylabel(ylabel_dict[key])	

				if (counter+1) == len(learning_parameters.keys()):
					ax[counter+1, n_index].set_xlabel("grad steps")

				counter = counter +1

	plt.savefig('Figures/graph_inference.png', format='png', dpi=100, bbox_inches='tight')

In [ ]:
# fig, ax = plt.subplots(2, len(learning_parameters), figsize = ((len(learning_parameters))*5, 10))

table_dict = {}
counter = 0
for key in learning_parameters.keys():
	table_dict[key] = []
	for j in range(learning_parameters[key]):
		optim_param_list = []
		for n_index in range(len(N_list)):
			N = N_list[n_index]

			output_path = "Data/GraphInference/Output/"+str(N)+"/"
			true_loss_file_name        = "graph_inference_"+str(N)+"_true_loss.npy"
			optim_loss_file_name       = "graph_inference_"+str(N)+"_optim_loss.npy"
			optim_parameters_file_name = "graph_inference_"+str(N)+"_optim_parameters.npy"

			true_loss        = np.load(output_path+true_loss_file_name,        allow_pickle=False)
			optim_loss       = np.load(output_path+optim_loss_file_name,       allow_pickle=False)
			optim_parameters = np.load(output_path+optim_parameters_file_name, allow_pickle=True)[()]

			index = np.unique(np.where(np.all(optim_loss!=0, axis = -1))[0])

			trial_index = np.nanargmin(optim_loss[index,...,-1], axis =1)
			to_append = np.stack([optim_parameters[key][trial,trial_index[trial],-1,j] for trial in index])
			# to_append = optim_parameters[key][2,:,-1,j]

			optim_param_list.append(tf.gather(to_append, tf.where(tf.math.is_nan(to_append)==False))[:,0])

		table_dict[key].append(optim_param_list)
		# ax[j,counter].boxplot(optim_param_list, showmeans=True)
		# ax[j,counter].axhline(parameters[key][j].numpy(), color = "red", alpha = 0.1)

	counter = counter +1

In [ ]:
# assign data
table = []

for key in learning_parameters.keys():
	for j in range(learning_parameters[key]):
		row = []
		row.append(ylabel_dict[key]+" "+str(np.round(parameters[key][j].numpy(), 3)))
		for n_index in range(len(N_list)):

			row.append(str(np.round(np.nanmean(table_dict[key][j][n_index]), 4))+"+/-"+str(np.round(np.nanstd(table_dict[key][j][n_index]), 4)))
	
		table.append(row)
 
# create header
head = ["Parameter", "N=500", "N=5000", "N=50000"]
 
# display table
print(tabulate(table, headers=head, tablefmt="latex_raw"))

# FM data

In [ ]:
learning_parameters = {#"logit_prior_infection":18053, 
		       "log_tau":1,
		       "log_delta":1, "log_zeta":1, "log_xi":1, 
		       "log_chi":1, "log_psi":1, "log_gamma":1,"log_epsilon":1}
	

ylabel_dict = {#"logit_prior_infection": r"$\text{logit}(\pi_{n,0})$", 
		"log_tau": r"$\log(\tau)$", 
		"log_delta":r"$\log(\delta)$", "log_zeta":r"$\log(\zeta)$", "log_xi":r"$\log(\xi)$", 
		"log_chi":r"$\log(\chi)$", "log_psi":r"$\log(\psi)$", "log_gamma":r"$\log(\gamma)$","log_epsilon":r"$\log(\epsilon)$"}


fig, ax = plt.subplots(int(len(learning_parameters.keys())/2), 2, figsize = (5*2, 2.5*int(len(learning_parameters.keys())/2)), sharex=True)

list_best_param = []
list_best_loss = []

best_initial = []

for test in range(10):
	output_path = "Data/FM/current/Output_"+str(test)+"/"
	loss_file_name       = "FM_loss.npy"

	loss       = np.load(output_path+loss_file_name,       allow_pickle=False)
	best_index = np.argmin(loss[:,-1])

	list_best_loss.append(loss[best_index,:])

	dict_best_param = {}

	for i in range(int(len(learning_parameters.keys())/2)):
		for j in range(2):

			key = list(learning_parameters.keys())[2*i+j]

			optim_parameters_file_name = "FM_parameters_"+key+".npy"
			optim_parameters = np.load(output_path+optim_parameters_file_name)[...,0]

			dict_best_param[key] = optim_parameters[-1,-1]

			ax[i,j].plot(optim_parameters[best_index,:], color = "blue")

			ax[i,j].set_ylabel(ylabel_dict[list(learning_parameters.keys())[2*i+j]])	

	key = "log_epsilon"#
	optim_parameters_file_name = "FM_parameters_"+key+".npy"
	optim_parameters = np.load(output_path+optim_parameters_file_name)[...,0]
	dict_best_param[key] = optim_parameters[-1,-1]

	list_best_param.append(dict_best_param)

	# plt.savefig('Figures/graph_inference.png', format='png', dpi=300)


In [ ]:
loss_sample = np.stack(list_best_loss)

parameters_list_sample = {}
for key in list_best_param[0].keys():
	parameters_list_sample[key] = []

	for i in range(len(list_best_param)):
		parameters_list_sample[key].append(list_best_param[i][key])

In [ ]:
parameters_sample = {}

for key in list_best_param[0].keys():
	
	parameters_sample[key] = np.stack(parameters_list_sample[key])

In [ ]:
best_sim = np.argmin(loss_sample[:,-1])

best_parameters = {}

for key in list_best_param[0].keys():
	best_parameters[key] = parameters_sample[key][best_sim]

In [ ]:
os.chdir("Scripts/")
%run -i FM_model.py
%run -i CAL.py
os.chdir("../")

In [ ]:
Y = tf.convert_to_tensor(np.load("Data/FM/Y_FM_cumbria.npy"), dtype = tf.float32)
indexes = tf.convert_to_tensor(np.load("Data/FM/indexes_FM_cumbria.npy"), dtype = tf.int64)
values  = tf.convert_to_tensor(np.load("Data/FM/values_FM_cumbria.npy"), dtype = tf.float32)
covariates = tf.convert_to_tensor(np.load("Data/FM/covariates_FM_cumbria.npy"), dtype = tf.float32)

In [ ]:
import time

parameters = {}
for key in best_parameters.keys():
	parameters[key] = tf.convert_to_tensor(best_parameters[key], dtype = tf.float32) 
parameters["logit_prob_testing"] = logit(tf.convert_to_tensor([0.0, 0.0, 1.0, 0.0], dtype = tf.float32))

FM_ibm = sparse_FM_SINR(values, indexes, covariates)

start = time.time()
loss = CAL_loss(FM_ibm, parameters, Y[:100])
print(time.time()-start)

In [ ]:
# Y_list = []
# for i in range(100):
# 	print(i)
# 	X, Y = FM_simulator(FM_ibm, parameters, 200)
# 	Y_list.append(tf.reduce_sum(Y, axis = 1)[:,3])

# np.save("Data/FM/Y_FM_list_stack.npy", np.stack(Y_list))

In [ ]:
best_loss_index = np.argmin(loss_sample[:,-1])

fig, ax = plt.subplots(1, 3, figsize = (15, 5))

ax[0].plot(loss_sample[best_loss_index,:])
ax[0].set_xlabel("iteration")
ax[0].set_ylabel("loss")


Y_cumbria = tf.convert_to_tensor(np.load("Data/FM/Y_FM_cumbria.npy"), dtype = tf.float32)
Y_list_stack = np.load("Data/FM/Y_FM_list_stack.npy")

time = np.linspace(1, 200, 200)
ax[2].plot(time, np.mean(Y_list_stack, axis = 0), color = "blue", )
ax[2].fill_between(time, np.quantile(Y_list_stack, (0.025, 0.975), axis = 0)[0,:], np.quantile(Y_list_stack, (0.05, 0.95), axis = 0)[1,:], color = "blue", alpha = 0.2)
ax[2].plot(time, tf.reduce_sum(Y_cumbria, axis = 1)[:200, 3], color = "blue", linestyle='dashed')
ax[2].set_xlabel("time")
ax[2].set_ylabel("notified")

Pi, Mu, Log_likelihood = CAL(FM_ibm, parameters, Y_cumbria)

ax[1].plot(time, tf.reduce_sum(Y_cumbria, axis = 1)[:200, 3], color = "blue", linestyle='dashed')
ax[1].plot(time, tf.reduce_sum(Mu, axis = 1)[:200,3], color = "blue", )

lower = tf.reduce_sum(Mu, axis = 1)[:200,3] - 1.96*tf.reduce_sum(Mu*(1-Mu), axis = 1)[:200,3] 
lower = tf.reduce_max(tf.stack((lower, tf.zeros(tf.shape(lower))), axis = -1), axis = -1)
upper = tf.reduce_sum(Mu, axis = 1)[:200,3] + 1.96*tf.reduce_sum(Mu*(1-Mu), axis = 1)[:200,3]
ax[1].fill_between(time, lower, upper, color = "blue", alpha=0.1)
ax[1].set_xlabel("time")
ax[1].set_ylabel("notified")

plt.savefig('Figures/FM_predictive.png', format='png', dpi=100)

In [ ]:
from tabulate import *

In [ ]:
# # assign data
# table = []

# farm = [[100, 0], [0, 1000], [50, 500], [2, 6]]

# for i in range(4):
# 	row = []
# 	row.append(farm[i][0])
# 	row.append(farm[i][1])
# 	row.append(np.exp(parameters["log_xi"])*(farm[i][0]**(np.exp(parameters["log_chi"]))) + (farm[i][1]**(np.exp(parameters["log_chi"]))))
# 	row.append(np.exp(parameters["log_zeta"])*(farm[i][0]**(np.exp(parameters["log_chi"]))) + (farm[i][1]**(np.exp(parameters["log_chi"]))))
# 	table.append(row)

# # create header
# head = ["Nr. cattle", "Nr. sheep", "Mean susceptibility", "Mean infectivity"]
 
# # display table
# print(tabulate(table, headers=head, tablefmt="latex_raw"))

In [ ]:
import seaborn as sns

def f(x, y):
    return np.exp(parameters["log_xi"])*(x**(np.exp(parameters["log_chi"]))) + (y**(np.exp(parameters["log_chi"])))

def h(x, y):
    return np.exp(parameters["log_zeta"])*(x**(np.exp(parameters["log_chi"]))) + (y**(np.exp(parameters["log_chi"])))

In [ ]:
x = np.linspace(0, 200, 100)
y = np.linspace(0, 2000, 100)
X, Y = np.meshgrid(x, y)
Z1 = f(X, Y)
Z2 = h(X, Y)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

sns.heatmap(Z1, ax=ax1, cmap='viridis', origin = "lower")
ax1.set_title('susceptibility')
ax1.set_xlabel('nr. cattle')  
ax1.set_ylabel('nr. sheep')  

# Set the x and y axis ticks to match x and y values
ax1.set_xticks(np.linspace(0, len(x)-1, num=5))  # 5 evenly spaced ticks
ax1.set_xticklabels(np.linspace(x.min(), x.max(), num=5).astype(int))
ax1.set_yticks(np.linspace(0, len(y)-1, num=5))  # 5 evenly spaced ticks
ax1.set_yticklabels(np.linspace(y.min(), y.max(), num=5).astype(int))

sns.heatmap(Z2, ax=ax2, cmap='plasma', origin = "lower")
ax2.set_title('infectivity')
ax2.set_xlabel('nr. cattle')  
ax2.set_ylabel('nr. sheep')  

# Set the x and y axis ticks to match x and y values
ax2.set_xticks(np.linspace(0, len(x)-1, num=5))  # 5 evenly spaced ticks
ax2.set_xticklabels(np.linspace(x.min(), x.max(), num=5).astype(int))
ax2.set_yticks(np.linspace(0, len(y)-1, num=5))  # 5 evenly spaced ticks
ax2.set_yticklabels(np.linspace(y.min(), y.max(), num=5).astype(int))

ax3.plot(np.linspace(0, 20, 21), np.exp(parameters["log_psi"])/(np.exp(2*parameters["log_psi"]) + np.linspace(0, 20, 21)*np.linspace(0, 20, 21))*np.where(np.linspace(0, 20, 21)>15, 0, 1))
ax3.set_xticks(np.linspace(0, 20, num=5))
ax3.set_xlabel("distance (Km)")
ax3.set_ylabel("spatial kernel")
fig.tight_layout()
plt.savefig('Figures/FM_susc_inf_spat.png', format='png', dpi=100)